---
layout: base
title: Hallpass
permalink: /bathroom/hallpass
menu: nav/toolkits/bathroom/menu.html
toc: false
---

{% include nav/toolkits/bathroom/menu.html %}

<div class="container">
    <div class="components">
        <!-- Resized Image -->
        <img src="https://github.com/user-attachments/assets/c350ab11-f560-4c76-b1f3-8bfbce017f77" alt="Teacher" style="max-width: 300px; height: auto; display: block; margin: 20px auto;">

        <p>Good job not using any paper, <span id="studentNameDisplay"></span>! You're saving both the environment and your seed!</p>

        <button type="button" onclick="window.approveStudent()">I'm done</button>
    </div>
</div>

<script type="module">
    import {javaURI, fetchOptions} from "{{site.baseurl}}/assets/js/api/config.js";

    // Get teacher name and student name from local storage or variables
    const teacherName = "jm1021@gmail.com"; // Replace with dynamic teacher name if needed

    const url = javaURI + `/api/queue`;
    const approveUrl = url + '/approve';
    const adminUrl = `${javaURI}/api/tinkle/add`;
    const postOptions = {
        ...fetchOptions,
        method: 'POST',
    };

    window.addEventListener('load', () => {
        fetchUser();
    });

    async function fetchUser() {
        const response = await fetch(javaURI + `/api/person/get`, {
            method: 'GET',
            cache: "no-cache",
            credentials: 'include',
            headers: { 
                'Content-Type': 'application/json',
                'X-Origin': 'client' 
            }
        });
        if (response.ok) {
            const userInfo = await response.json();
            window.studentName = userInfo.name;
            document.getElementById("studentNameDisplay").textContent = window.studentName;
            console.log("Person: ", userInfo);
        }
    }
    

    // Function to approve student who is first in line
    window.approveStudent = function() {
        const now = new Date();
        const hours = now.getHours();
        const minutes = now.getMinutes().toString().padStart(2, "0"); // Ensures 2 digits
        const seconds = now.getSeconds().toString().padStart(2, "0");
        const timeOut = `${hours}:${minutes}:${seconds}`;
        const timeIn = localStorage.getItem("timeIn");
        var timeSpentMinutes = 0;

        if (timeIn) {
            // Convert both timeIn and timeOut from "HH:MM:SS" to total seconds
            const [inHours, inMinutes, inSeconds] = timeIn.split(":").map(Number);
            const [outHours, outMinutes, outSeconds] = timeOut.split(":").map(Number);

            // Convert both times to total seconds since midnight
            const totalSecondsIn = inHours * 3600 + inMinutes * 60 + inSeconds;
            const totalSecondsOut = outHours * 3600 + outMinutes * 60 + outSeconds;

            // Calculate the difference in seconds
            const timeSpentSeconds = totalSecondsOut - totalSecondsIn;

            // Convert timeSpent from seconds to minutes (with 2 decimal places)
            const timeSpentMinutes = (timeSpentSeconds / 60).toFixed(2);

            console.log(`Time spent: ${timeSpentMinutes} minutes`);
        } else {
            console.log("No timeIn value stored in localStorage.");
        }

        const timeInOut = `${timeIn}-${timeOut}`;

        const times = {
            studentEmail: studentName,
            timeIn: timeInOut,
            averageDuration: timeSpentMinutes,
        };

        const queuer = {
            teacherEmail: teacherName,
            studentName: studentName,
            uri: javaURI,
        };

        fetch(adminUrl, {
            ...postOptions,
            body: JSON.stringify(times),
        })
            .then(response => {
                if (response.ok) {
                    console.log("Time added to database");
                } else {
                    alert("Failed to add time to database.");
                }
            })
            .catch(error => console.error("Error adding time to database:", error));

        fetch(approveUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => {
                if (response.ok) {
                    console.log("Successful Approval");
                } else {
                    alert("Failed to approve student.");
                }
            })
            .catch(error => console.error("Error approving student:", error));
        window.location.href = "{{site.baseurl}}/profile";
    }
</script>